In [1]:
import pandas as pd

data = {
    'Mixture': ['Caf', 'Ace', 'Asp'],
    'Boiling_temp_(°C)': [178, 332.7, 246],
    'logP': [-0.07, -1.33, 0.07],
    'pKa': [[14], [3.02], [1.83, 9.13]],
    'Molecular_mass': [194, 204, 133],
}

df = pd.DataFrame(data)

def det_chromato(df):
    if df['Boiling_temp_(°C)'].min() >= 300:
        Chromato_type = 'GC'
        eluent_nature = 'gas'
        proposed_pH = None
    else:
        max_molar_mass = df['Molecular_mass'].max()
        min_pKa = float('inf')  # Initialize min_pKa with a large value
        max_pKa = float('-inf')  # Initialize max_pKa with a small value
        for pKa_entry in df['pKa']:
            if isinstance(pKa_entry, list):
                for pKa_value in pKa_entry:
                    min_pKa = min(pKa_value, min_pKa)
                    max_pKa = max(pKa_value, max_pKa)
            else:
                min_pKa = min(pKa_entry, min_pKa)
                max_pKa = max(pKa_entry, max_pKa)
        
        if max_molar_mass <= 2000:
            max_logP = df['logP'].max()
            min_logP = df['logP'].min()
            if max_logP < 0:
                proposed_pH = max_pKa + 2
                if 3 <= proposed_pH <= 11 and max_pKa + 2 >= proposed_pH:
                    Chromato_type = 'IC'
                    eluent_nature = 'aqueous'
                    proposed_pH = max_pKa + 2
                else:
                    Chromato_type = 'HPLC'
                    eluent_nature = 'organic or hydro-organic'
                    proposed_pH = min_pKa + 2
            else:
                Chromato_type = 'HPLC'
                if -2 <= min_logP <= 0:
                    eluent_nature = 'organic or hydro-organic'
                    if min_logP >= 0:
                        Chromato_type += ' on normal stationary phase'
                    else:
                        Chromato_type += ' on reverse stationary phase using C18 column'
                else:
                    eluent_nature = 'organic or hydro-organic'
                    Chromato_type += ' on normal stationary phase'
                proposed_pH = min_pKa + 2
        else:
            max_logP = df['logP'].max()
            min_logP = df['logP'].min()
            if max_logP < 0:
                Chromato_type = 'HPLC on reverse stationary phase'
                eluent_nature = 'organic or hydro-organic'
                proposed_pH = min_pKa + 2
            else:
                if max_logP > 0:
                    Chromato_type = 'SEC on gel permeation with a hydrophobe organic polymer stationary phase'
                    eluent_nature = 'organic solvent'
                else:
                    Chromato_type = 'SEC on gel filtration with a polyhydroxylated hydrophile polymer stationary phase'
                    eluent_nature = 'aqueous'
                proposed_pH = min_pKa + 2
    
    return Chromato_type, eluent_nature, proposed_pH

Mixture_chromato_type, eluent_nature, proposed_pH = det_chromato(df)

print(f"The advisable chromatography type is: {Mixture_chromato_type}")
print(f"Eluent nature: {eluent_nature}")
if proposed_pH is not None:
    print(f"Proposed pH for the eluent: {proposed_pH}")


The advisable chromatography type is: HPLC on reverse stationary phase using C18 column
Eluent nature: organic or hydro-organic
Proposed pH for the eluent: 3.83
